#  Exerciese 6-2

## Myokyung Han (20224230)

# Data Downloading

### Exchange Rate - krw_usd_xr.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3A036Y001.0000001

### Current Account - Korea - current_account_korea.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3A022Y013.000000

### Current Account - US - current_account_us.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y031.US

### GDP: Korea - gdp_korea.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y042.KOR

### GDP Growth korea - gdp_growth_korea.xlsx 
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3A901Y001.AI1AA
    
### GDP: US - gdp_us.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y042.USA

### GDP Growth US - gdp_growth_us.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y041.USA

### Consumer Price Index: Korea - cpi_korea.xlsx >> non stationary
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y022.KR

### Consumer Price Index: US - cpi_us.xlsx >> non stationary
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3AI10Y022.US

### 1-year Interest Rate: Korea - int_rate_korea.xlsx
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3A060Y001.010190000
m
### 1-year Interest Rate: US - int_rate_us.xlsx

https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=FRED%3ADGS1

### Foreign Reserves at Bank of Korea - foreign_reserves.xlsx >> non stationary
https://www.deepsearch.com/analytics/economic-indicator?pageView=1&symbol=BOK%3A901Y001.AI1DC

## Data Loading

In [1]:
import pandas as pd

In [2]:
def load_deepsearch_data(filepath, column):
    df = pd.read_excel(filepath)
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.set_index('날짜')
    df.columns = [column]
    
    return df

In [3]:
filenames = [
    'krw_usd_xr',
    'current_account_korea', 'current_account_us',
    'gdp_korea', 'gdp_us',
    'gdp_growth_korea', 'gdp_growth_us',
    'cpi_korea', 'cpi_us',
    'int_rate_korea', 'int_rate_us',
    'foreign_reserves',
]

In [4]:
dfs = []
for fname in filenames:
    print('Loading {}...'.format(fname))
    # 짱신기해...
    # filepath = './data/{}.xlxs'.format(fname)와 동일
    filepath = f'./data/{fname}.xlsx'
    
    df = load_deepsearch_data(filepath, fname)
    
    if fname in ['gdp_growth_korea', 'gdp_growth_us', 'current_account_us']:
        df[fname] = df[fname].rolling(4).sum()
    elif fname in ['current_account_korea']:
        df[fname] = df[fname].rolling(12).sum()
        
    
    dfs.append(df)
    
print('Data loading is completed')

Loading krw_usd_xr...
Loading current_account_korea...
Loading current_account_us...
Loading gdp_korea...
Loading gdp_us...
Loading gdp_growth_korea...
Loading gdp_growth_us...
Loading cpi_korea...
Loading cpi_us...
Loading int_rate_korea...
Loading int_rate_us...
Loading foreign_reserves...
Data loading is completed


In [5]:
from pandas.tseries.offsets import YearEnd

In [6]:
def load_world_bank_data(filepath, column):
    df = pd.read_excel(filepath, sheet_name = 'Data', skiprows = 3)
    
    df = df.set_index('Country Code')
    df = df.loc['KOR'].to_frame()
    
    df = df.iloc[3:]
    df.dropna(inplace=True)
    df['KOR'] = df['KOR'].astype(float)
    df['date'] = pd.to_datetime(df.index)+YearEnd(0)
    df.set_index('date', inplace=True)
    df.columns = [column]
    return df
      
    

In [7]:
df_ppp_xr = load_world_bank_data('./data/world_ppp_xr.xls', 'krw_usd_ppp_xr')

In [8]:
dfs.append(df_ppp_xr)

In [9]:
len(dfs)

13

In [10]:
df_merged = pd.concat(dfs, axis = 1)

In [11]:
df_merged = df_merged.resample('M').last()

In [12]:
df_merged = df_merged.fillna(method='ffill')

In [13]:
df_merged['2010-01-01':].head()

,krw_usd_xr,current_account_korea,current_account_us,gdp_korea,gdp_us,gdp_growth_korea,gdp_growth_us,cpi_korea,cpi_us,int_rate_korea,int_rate_us,foreign_reserves,krw_usd_ppp_xr
2010-01-31,1156.5,31135.4,-372522.0,943941.9,14448932.0,5.1,0.3,98.82,99.37,3.22,0.30,273693584.0,828.420848
2010-02-28,1158.4,25800.0,-372522.0,943941.9,14448932.0,7.0,0.3,99.02,99.40,3.07,0.32,270659721.0,828.420848
2010-03-31,1130.8,20931.1,-382165.0,943941.9,14448932.0,7.0,1.8,99.22,99.81,2.70,0.41,272331061.0,828.420848
2010-04-30,1115.5,18588.8,-382165.0,943941.9,14448932.0,7.0,1.8,99.62,99.98,2.65,0.41,278873477.0,828.420848
2010-05-31,1200.2,22909.8,-382165.0,943941.9,14448932.0,7.6,1.8,99.72,100.06,2.75,0.34,270221038.0,828.420848


In [14]:
df_merged['2010-01-01':].tail()

,krw_usd_xr,current_account_korea,current_account_us,gdp_korea,gdp_us,gdp_growth_korea,gdp_growth_us,cpi_korea,cpi_us,int_rate_korea,int_rate_us,foreign_reserves,krw_usd_ppp_xr
2021-12-31,1185.5,90382.5,-783982.0,1637895.8,20893746.0,3.7,4.8,120.45,127.86,1.350,0.39,463118362.0,824.604924
2022-01-31,1202.4,85133.4,-783982.0,1637895.8,20893746.0,3.7,4.8,121.21,128.93,1.463,0.78,461526656.0,824.604924
2022-02-28,1202.7,85133.4,-783982.0,1637895.8,20893746.0,3.7,4.8,121.21,128.93,1.503,1.01,461766293.0,824.604924
2022-03-31,1210.8,85133.4,-783982.0,1637895.8,20893746.0,3.7,4.8,121.21,128.93,1.815,1.63,461766293.0,824.604924
2022-04-30,1216.7,85133.4,-783982.0,1637895.8,20893746.0,3.7,4.8,121.21,128.93,1.852,1.63,461766293.0,824.604924


## Compute new variables

### Currency returens

In [15]:
df_merged['ann_ret']  = df_merged['krw_usd_xr'] .pct_change(periods=12)*100

In [16]:
df_merged['F12.ann_ret'] = df_merged['ann_ret'].shift(-12)

### Real XR

In [17]:
df_merged['real_xr'] = df_merged['krw_usd_ppp_xr']  /df_merged['krw_usd_xr']  

### current account relative to GDP
non_stationary!!

In [18]:
df_merged['ca_gdp_korea'] = df_merged['current_account_korea']/df_merged['gdp_korea']*100

In [19]:
df_merged['ca_gdp_us'] = df_merged['current_account_us']/df_merged['gdp_us']*100

In [20]:
df_merged.loc['2019-10-01':][['ca_gdp_korea', 'ca_gdp_us']].head()

,ca_gdp_korea,ca_gdp_us
2019-10-31,3.470373,-2.601153
2019-11-30,3.519438,-2.601153
2019-12-31,3.651917,-2.325728
2020-01-31,3.512475,-2.325728
2020-02-29,3.665722,-2.325728


### GDP Growth

In [21]:
df_merged['gdp_growth_korea2'] = df_merged['gdp_korea'].pct_change(periods=12)*100

In [22]:
df_merged['gdp_growth_us2'] = df_merged['gdp_us'].pct_change(periods=12)*100

In [26]:
df_merged.loc[['2018-12-31', '2019-12-31']][['gdp_us', 'gdp_growth_us','gdp_growth_us2']]

,gdp_us,gdp_growth_us,gdp_growth_us2
2018-12-31,20580223.0,2.5,5.434581
2019-12-31,21427700.0,2.3,4.117919


In [27]:
# why is it differnece!!!!
# https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?locations=US
# Current US 달러이냐 constant us dollar이냐에 따라 달라진다. 
# Current US>> market
# constant us dollar >> inflation.... 2015년 기준으로 보았을때 2.5가 된다..

### Inflation

In [30]:
df_merged['infl_korea'] = df_merged['cpi_korea'].pct_change(periods=12)*100

In [31]:
df_merged['infl_us'] = df_merged['cpi_us'].pct_change(periods=12)*100

### Inflation Interest rate differential

In [32]:
df_merged['infl_diff'] = df_merged['infl_korea']- df_merged['infl_us']

In [33]:
df_merged['int_rate_diff'] = df_merged['int_rate_korea']- df_merged['int_rate_us']

### Foreign Reserves

In [35]:
df_merged['reserves_to_gdp'] = df_merged['foreign_reserves']/df_merged['gdp_korea']/1000*100
# 1000 >>단위때문에 곱하고

In [36]:
df_merged['reserves_growth'] = df_merged['foreign_reserves'].pct_change(periods=12)*100
# 다시 들어야겠따...
# trand how it is changing ......